<a href="https://colab.research.google.com/github/AndrewMarksArt/DS-Unit-2-Applied-Modeling/blob/master/Log_Linear_Regression_Assigment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import category_encoders as ce

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error

In [2]:
!wget https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/caterpillar/caterpillar-tube-pricing.zip

--2019-08-12 18:59:33--  https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/caterpillar/caterpillar-tube-pricing.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 823789 (804K) [application/zip]
Saving to: ‘caterpillar-tube-pricing.zip.2’

caterpillar-tube-pr 100%[===================>] 804.48K  --.-KB/s    in 0.04s   

2019-08-12 18:59:33 (20.1 MB/s) - ‘caterpillar-tube-pricing.zip.2’ saved [823789/823789]



In [3]:
!unzip caterpillar-tube-pricing.zip

Archive:  caterpillar-tube-pricing.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: sample_submission.csv   
  inflating: data.zip                


In [4]:
!unzip data.zip

Archive:  data.zip
replace competition_data/bill_of_materials.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: competition_data/bill_of_materials.csv  
  inflating: competition_data/comp_adaptor.csv  
  inflating: competition_data/comp_boss.csv  
  inflating: competition_data/comp_elbow.csv  
  inflating: competition_data/comp_float.csv  
  inflating: competition_data/comp_hfl.csv  
  inflating: competition_data/comp_nut.csv  
  inflating: competition_data/comp_other.csv  
  inflating: competition_data/comp_sleeve.csv  
  inflating: competition_data/comp_straight.csv  
  inflating: competition_data/comp_tee.csv  
  inflating: competition_data/comp_threaded.csv  
  inflating: competition_data/components.csv  
  inflating: competition_data/specs.csv  
  inflating: competition_data/test_set.csv  
  inflating: competition_data/train_set.csv  
  inflating: competition_data/tube.csv  
  inflating: competition_data/tube_end_form.csv  
  inflating: competition_data/type_component.csv 

In [5]:
from glob import glob
import pandas as pd

for path in glob('competition_data/*.csv'):
    df = pd.read_csv(path)
    print(path, df.shape)

competition_data/comp_tee.csv (4, 14)
competition_data/type_component.csv (29, 2)
competition_data/bill_of_materials.csv (21198, 17)
competition_data/tube_end_form.csv (27, 2)
competition_data/comp_threaded.csv (194, 32)
competition_data/comp_sleeve.csv (50, 10)
competition_data/comp_nut.csv (65, 11)
competition_data/type_connection.csv (14, 2)
competition_data/comp_other.csv (1001, 3)
competition_data/comp_elbow.csv (178, 16)
competition_data/type_end_form.csv (8, 2)
competition_data/components.csv (2048, 3)
competition_data/train_set.csv (30213, 8)
competition_data/comp_boss.csv (147, 15)
competition_data/comp_adaptor.csv (25, 20)
competition_data/specs.csv (21198, 11)
competition_data/tube.csv (21198, 16)
competition_data/comp_float.csv (16, 7)
competition_data/comp_hfl.csv (6, 9)
competition_data/comp_straight.csv (361, 12)
competition_data/test_set.csv (30235, 8)


In [0]:
def rmsle(y_true, y_pred):
  return np.sqrt(mean_squared_log_error(y_true, y_pred))

In [0]:
def rmse(y_true, y_pred):
  return np.sqrt(mean_squared_error(y_true, y_pred))

In [0]:
def wrangle(X):
  X = X.copy()
  
  # engineer date features
  X['quote_date'] = pd.to_datetime(X['quote_date'], infer_datetime_format=True)
  X['quote_date_year'] = X['quote_date'].dt.year
  X['quote_date_month'] = X['quote_date'].dt.month
  X = X.drop(columns='quote_date')
  
  # merge tube data
  tube = pd.read_csv('competition_data/tube.csv')
  specs = pd.read_csv('competition_data/specs.csv')
  bill_mats = pd.read_csv('competition_data/bill_of_materials.csv')
  
  X = X.merge(tube, how='left')
  X = X.merge(specs, how='left')
  X = X.merge(bill_mats, how='left')
  
  
  # drop tube_assembly_id because our goal is to predict unknow assemblies
  X = X.drop(columns='tube_assembly_id')
  
  return X

In [49]:
bill_mats.shape

(21198, 17)

In [0]:
trainval = pd.read_csv('competition_data/train_set.csv')
test = pd.read_csv('competition_data/test_set.csv')

In [0]:
trainval['quote_date'] = pd.to_datetime(trainval['quote_date'], infer_datetime_format=True)
test['quote_date'] = pd.to_datetime(test['quote_date'], infer_datetime_format=True)

In [25]:
trainval_tube_assemblies = trainval['tube_assembly_id'].unique()
test_tube_assemblies = test['tube_assembly_id'].unique()
len(trainval_tube_assemblies), len(test_tube_assemblies)

(8855, 8856)

In [0]:
trainval_tube_assemblies, val_tube_assemblies = train_test_split(
    trainval_tube_assemblies, random_state=42
)

In [27]:
train = trainval[trainval['tube_assembly_id'].isin(trainval_tube_assemblies)]
val = trainval[trainval['tube_assembly_id'].isin(val_tube_assemblies)]

train.shape, val.shape, trainval.shape

((22628, 8), (7585, 8), (30213, 8))

In [28]:
train_wrangled = wrangle(train)
val_wrangled = wrangle(val)

train_wrangled.shape, val_wrangled.shape

((22628, 49), (7585, 49))

In [29]:
target = 'cost'
features = train_wrangled.columns.drop(target)
print('Features: ', features.tolist())

Features:  ['supplier', 'annual_usage', 'min_order_quantity', 'bracket_pricing', 'quantity', 'quote_date_year', 'quote_date_month', 'material_id', 'diameter', 'wall', 'length', 'num_bends', 'bend_radius', 'end_a_1x', 'end_a_2x', 'end_x_1x', 'end_x_2x', 'end_a', 'end_x', 'num_boss', 'num_bracket', 'other', 'spec1', 'spec2', 'spec3', 'spec4', 'spec5', 'spec6', 'spec7', 'spec8', 'spec9', 'spec10', 'component_id_1', 'quantity_1', 'component_id_2', 'quantity_2', 'component_id_3', 'quantity_3', 'component_id_4', 'quantity_4', 'component_id_5', 'quantity_5', 'component_id_6', 'quantity_6', 'component_id_7', 'quantity_7', 'component_id_8', 'quantity_8']


In [0]:
X_train = train_wrangled[features]
X_val = val_wrangled[features]

In [0]:
y_train = train[target]
y_val = val[target]
y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)

In [0]:
X_train = X_train.fillna('0')
X_val = X_val.fillna('0')

In [56]:
pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
)

pipeline.fit(X_train, y_train_log)

Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['supplier', 'bracket_pricing',
                                      'material_id', 'end_a_1x', 'end_a_2x',
                                      'end_x_1x', 'end_x_2x', 'end_a', 'end_x',
                                      'spec1', 'spec2', 'spec3', 'spec4',
                                      'spec5', 'spec6', 'spec7', 'spec8',
                                      'spec9', 'spec10', 'component_id_1',
                                      'quantity_1', 'component_id_2',
                                      'quantity_2', 'component_id_3',
                                      'quantity_3', 'componen...
                ('randomforestregressor',
                 RandomForestRegressor(bootstrap=True, criterion='mse',
                                       max_depth=None, max_features='auto',
                                       max_leaf_nodes=None,
                                       

In [59]:
y_pred_log = pipeline.predict(X_val)
rmse(y_val_log, y_pred_log)

0.30374109665266746

In [60]:
test_wrangled = wrangle(test)
X_test = test_wrangled.drop(columns='id')
all(X_test.columns == X_train.columns)

True

In [0]:
y_pred_log = pipeline.predict(X_test)
y_pred = np.expm1(y_pred_log)

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission['cost'] = y_pred
submission.to_csv('log_assignment_submission-01.csv', index=False)